# Azure AI Agents MCP Example

This notebook demonstrates how to create and interact with an Azure AI Foundry Agent using the Azure AI Projects SDK. Supported regions at time of writing: westus, westus2, uaenorth, southindia and switzerlandnorth

In [17]:
import time
import json
from dotenv import load_dotenv
import os

# Load environment variables from a .env file
load_dotenv()

True

In [18]:
from azure.ai.agents.models import MessageTextContent, ListSortOrder
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=os.getenv("PROJECT_ENDPOINT2"),
    credential=DefaultAzureCredential(),
)

In [19]:
# Create an agent with MCP tool support
agent = project_client.agents.create_agent(
    model=os.getenv("MODEL_DEPLOYMENT_NAME"),
    name="my-mcp-agent",
    instructions=(
        "You are a helpful assistant. Use the tools provided "
        "to answer the user's questions. Be sure to cite your sources."
    ),
    tools=[{
        "type": "mcp",
        "server_label": "MicrosoftDocs",
        "server_url": "https://learn.microsoft.com/api/mcp",
        "require_approval": "never"
    }],
    tool_resources=None
)
print(f"Created agent, agent ID: {agent.id}")


Created agent, agent ID: asst_bkaLGyf7y6cf3jjID84QMPGe


In [20]:
# Start a conversation thread and send a user message
thread = project_client.agents.threads.create()
print(f"Created thread, thread ID: {thread.id}")

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Does Azure AI Foundry Agent Service support Model Context Protocol as a tool?"
)
print(f"Created message, message ID: {message.id}")


Created thread, thread ID: thread_jFQIyZpteIvNNY7LAnzqbW8L
Created message, message ID: msg_FmncGTRCNBm8tepFfeziuzUr


In [21]:
# Run the agent and poll for completion
run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)

# Poll the run status
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

if run.status == "failed":
    print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [22]:
# Inspect run steps for tool calls
run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Run step: {step.id}, status: {step.status}, type: {step.type}")
    if step.type == "tool_calls":
        print("Tool call details:")
        for tool_call in step.step_details.tool_calls:
            print(json.dumps(tool_call.as_dict(), indent=2))

Run step: step_ORIdBzvZhjoEKwDfwKrXFi6r, status: RunStepStatus.COMPLETED, type: RunStepType.MESSAGE_CREATION
Run step: step_MuRVt2gKixRMbxpn7grbjE7O, status: RunStepStatus.COMPLETED, type: RunStepType.TOOL_CALLS
Tool call details:
{
  "id": "call_Cdrk5Xya8FE2WTC8h3tF5Ynk",
  "type": "mcp",
  "arguments": "{\"question\":\"Does Azure AI Foundry Agent Service support Model Context Protocol as a tool?\"}",
  "name": "microsoft_docs_search",
  "output": "[{\"title\": \"What's new in Azure AI Foundry Agent Service\", \"content\": \"# What's new in Azure AI Foundry Agent Service\\nThis article provides a summary of the latest releases and major documentation updates for Azure AI Foundry Agent Service.\\n## June 2025\\n### Model Context Protocol (MCP) tool\\nYou can new extend the capabilities of your agents by connecting them to tools hosted on remote [Model Context Protocol](https://modelcontextprotocol.io/introduction) (MCP) servers by using the [MCP tool](https://learn.microsoft.com/en-us/

In [23]:
# List all messages in the thread, printing assistant responses
messages = project_client.agents.messages.list(
    thread_id=thread.id, order=ListSortOrder.ASCENDING
)
for data_point in messages:
    last_content = data_point.content[-1]
    if isinstance(last_content, MessageTextContent):
        print(f"{data_point.role}: {last_content.text.value}")

MessageRole.USER: Does Azure AI Foundry Agent Service support Model Context Protocol as a tool?
MessageRole.AGENT: Yes, Azure AI Foundry Agent Service supports Model Context Protocol (MCP) as a tool. You can extend the capabilities of your agents by connecting them to tools hosted on remote Model Context Protocol (MCP) servers using the built-in MCP tool. This allows your agent to interact with external tools and APIs via the MCP standard, which is designed for scalable integration of external functions into AI workflows.

Key points:
- The MCP tool lets you add external MCP server endpoints to your agent, making their functions callable by the agent.
- The service supports custom headers (such as authentication tokens) when connecting to MCP servers.
- Documentation and step-by-step setup instructions are available to help you enable the MCP tool for your agents.

References:
- What's new in Azure AI Foundry Agent Service (June 2025 includes MCP tool): [Learn more](https://learn.micro

In [24]:
# Clean up by deleting the agent
project_client.agents.delete_agent(agent.id)
print(f"Deleted agent, agent ID: {agent.id}")

Deleted agent, agent ID: asst_bkaLGyf7y6cf3jjID84QMPGe
